In [123]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import os
import numpy as np
import gc
import time

In [146]:
class TeemoModelView:
    def __init__(self, webdriver_path='C:\WebDrivers\edgedriver_win64\msedgedriver.exe', champ_name='Teemo', init=False):
        self.wd_path = webdriver_path
        self.driver = webdriver.Edge(executable_path=self.wd_path)
        self.champ_name = champ_name
        self.initd = False
        self.item_ids = {'champ_sel'      : 'teemo-mv-objects',
            'champ_skin_sel' : 'teemo-mv-skins',
            'load_btn'       :'teemo-mv-load-button',
            'animation_sel'  : 'model-viewer-animation',
            'color_opts': 'adv-opts-bg-color',
            'anim_speed_txt' : 'model-viewer-animation-speed',
            'zoom_speed' : 'model-viewer-zoom-speed',
            'model_viewer': 'champion-model'
           }
        self.js_fxns = {'play_pause' : 'toggleAnimationPlay()',
                   'toggle_adv_options' : 'toggleAdvancedOptions()',
                   'full_scrn': 'makeFullscreen()'
                  }
        if(init):
            self.init_driver()
              
    def init_driver(self, wait_for_load=True, wait_to=20):
        self.driver.get(self.mv_prep_query(champ_name=self.champ_name))
        self._init_elements()
        self.load_champ(self.champ_name, wait_for_load=wait_for_load,wait_to=wait_to) 
        self.initd = True
        
    def _init_elements(self):
        self.element_champ_sel = self.driver.find_element_by_id(self.item_ids['champ_sel'])
        self.element_champ_skin_sel = self.driver.find_element_by_id(self.item_ids['champ_skin_sel'])
        self.element_champ_anim_sel = self.driver.find_element_by_id(self.item_ids['animation_sel'])
        self.element_model_viewer =  self.driver.find_element_by_id(self.item_ids['model_viewer'])
        self.element_btn_load = self.driver.find_element_by_id(self.item_ids['load_btn'])
        self.champ_names = self.get_champ_names()
        
    def mv_prep_query(self, champ_name='Teemo', skin_id= '0', game='league-of-legends', model_category='champions'):
        url = 'https://teemo.gg/model-viewer?'
        url = url + 'game=' + game + '&'
        url = url + 'type='+ model_category + '&'
        url = url + 'object='+ champ_name + '&'
        url = url + 'skin_id='+ champ_name + '-' + skin_id
        return url
    
    def get_champ_names(self):
        #element_sel = self.driver.find_element_by_id(item_ids['champ_sel'])
        return self.element_champ_sel.text.split('\n')
    
    def get_champ_skins(self):
        #element_sel = self.driver.find_element_by_id(item_ids['champ_skin_sel'])
        return self.element_champ_skin_sel.text.split('\n')
    
    def get_champ_animations(self):
        #element_sel = self.driver.find_element_by_id(item_ids['animation_sel'])
        return [a.strip() for a in self.element_champ_anim_sel.text.split('\n')]
    
    def load_champ(self, champ_name='Sona', wait_for_load=True, wait_to=20):  
        assert champ_name in self.champ_names, "Champion {} was not found".format(champ_name)
        #element_sel = self.driver.find_element_by_id(item_ids['champ_sel'])
        champ_sel = Select(self.element_champ_sel)
        champ_sel.select_by_visible_text(champ_name)
        self.champ_name = champ_name
        self.element_btn_load.click()
        if(wait_for_load and wait_to>0):
            wait_txt = 'eath'
            self.wait_for_model_load(wait_txt, wait_to)
        self._init_elements()
    
    def load_champ_skin(self, champ_skin, wait_for_load=True, wait_to=20):
        assert champ_skin in get_champ_skins(driver), "Champion skin {} was not found".format(champ_skin)
        #element_sel = driver.find_element_by_id(item_ids['champ_skin_sel'])
        champ_sel = Select(self.element_champ_skin_sel)
        champ_sel.select_by_visible_text(champ_skin)
        self.element_btn_load.click()
        if(wait_for_load and wait_to>0):
            wait_txt = 'eath'
            self.wait_for_model_load(wait_txt, wait_to)
        self._init_elements()
        
    def set_champ_animation_by_name(self, anim_name):
        assert anim_name in self.get_champ_animations(), "Champion anim {} was not found".format(anim_name)
        #element_sel = driver.find_element_by_id(item_ids['animation_sel'])
        sel = Select(self.element_champ_anim_sel)
        sel.select_by_visible_text(anim_name)
    
    def set_champ_animation_by_idx(self, anim_idx):
        n_anim = len(self.get_champ_animations())
        assert anim_idx < n_anim, "Index {} is greater than animations array length {}".format(anim_idx,n_anim)
        #element_sel = driver.find_element_by_id(item_ids['animation_sel'])
        sel = Select(self.element_champ_anim_sel)
        sel.select_by_index(anim_idx)
    
    def take_model_screenshot(self, save_name):
        #assert os.path.exists(save_name)
        #self.driver.find_element_by_id(item_ids['model_viewer']).click()
        self.element_model_viewer.screenshot(save_name)
        
    def key_down_with_delay(self, key, press_delay=0.25):
        mv = self.element_model_viewer
        keyAction = ActionChains(self.driver).key_down(key,element=mv).pause(press_delay).key_up(key)
        keyAction.perform()
    
    def set_model_backgroud(self, color = '#00ff00'):
        self.driver.execute_script('\
        background&&background.dispose();\
        scene.clearColor=new BABYLON.Color3.FromHexString("{}")'.format(color))
    
    def get_curr_anime_max(self):
        return float(self.driver.find_element_by_id('model-viewer-frame').get_attribute('max'))
    
    def set_model_animation_pos(self, val):
        assert val < self.get_curr_anime_max()
        assert val >=0
        js = 'pauseAnimation();currentAnimation&&currentAnimation.goToFrame({})'.format(val)
        self.driver.execute_script(js)
        
    def set_model_min_max_camera_radii(self, minimum=600, maximum=700):
        self.driver.execute_script('scene.activeCamera.upperRadiusLimit={};\
                                    scene.activeCamera.lowerRadiusLimit={};'.format(maximum, minimum))
        
    def wait_for_model_load(self, wait_txt, timeout=20):
        wait = WebDriverWait(self.driver,timeout)
        el = wait.until(EC.text_to_be_present_in_element((By.ID, self.item_ids['animation_sel']), wait_txt))
        return el

In [126]:
champs = ['Garen', 'Nasus', 'Morgana']
key_wrds = ['ttack', 'eath', 'dle','pell' ]
view_angles = [0,45,90,135,180,225,270,315,360]
save_dir = 'champions'

In [ ]:
mv = TeemoModelView(champ_name=champs[0], init=False)
for champ in champs:
    if(not mv.initd):
        mv.init_driver()
    mv.load_champ(champ)
    mv.set_model_min_max_camera_radii(minimum=1000, maximum=1100)
    mv.set_model_backgroud('#00ff00') # Green
    
    # Change the models orientation to isometrc-like view
    mv.key_down_with_delay(key=Keys.ARROW_UP,press_delay=0.3)
    
    # Get all available animations, filter to get those we want
    anim_names = [an_name.strip() for k in key_wrds for an_name in mv.get_champ_animations() if k in an_name]
   
    for anim in anim_names:
        # MAke save path
        save_path = '{}/{}/anim_{}'.format(save_dir,champ, anim)
        if(not os.path.exists(save_path)):
            os.makedirs(save_path)
            
        mv.set_champ_animation_by_name(anim)
        an_max = mv.get_curr_anime_max()
        # Create animation frames with 0.1s intervals (10 fps)
        anim_pints = np.arange(0.0, float(an_max), 0.1) 
        #mv.set_model_min_max_camera_radii()
        for an_ctr, anim_pnt in zip(range(0, len(anim_pints)), anim_pints):
            # Set anim frame
            mv.set_model_animation_pos(anim_pnt)
            
            # Take screenshots of this animation from various angles
            for direxn in range(0, 8):
                file_nm = '{}_{}_f{}_a{}.png'.format(champ, anim, an_ctr, direxn)
                full_path = os.path.join(save_path, file_nm)
                mv.take_model_screenshot(full_path)
                
                # Rotate the model horizontally
                mv.key_down_with_delay(key=Keys.ARROW_LEFT,press_delay=0.25)

In [119]:
gc.collect()

9626

In [132]:
mv = TeemoModelView(champ_name=champs[0], init=True)

In [134]:
mv.set_model_min_max_camera_radii()
   # mv.key_down_with_delay(key=Keys.ARROW_UP, press_delay=0.25)

In [149]:
driver = mv.driver

In [145]:
driver.execute_script('scene.activeCamera.upperRadiusLimit= 700;')
driver.execute_script('scene.activeCamera.lowerRadiusLimit=600;')

In [150]:
mv_el = mv.element_model_viewer

In [151]:
mv_el.value_of_css_property('width')

'912px'

In [77]:
mv.set_champ_animation_by_name(anim_names[2])

In [58]:
anim_names[0].strip() in mv.get_champ_animations()

False

In [59]:
anim_names[0].strip()

'Garen_2013_Attack_01'

In [68]:
[el.strip() for el in mv.element_champ_anim_sel.text.split('\n')]

['Animation',
 'T Pose',
 'Garen_2013_Attack_01',
 'Garen_2013_Attack_02',
 'Garen_2013_Crit',
 'Garen_2013_Death',
 'Garen_2013_Idle1',
 'Garen_2013_Idle2',
 'Garen_2013_Idle3',
 'Garen_2013_Idle_LeadIn',
 'Garen_2013_Joke',
 'Garen_2013_Joke_loop',
 'Garen_2013_Recall',
 'Garen_2013_Recall_leadout',
 'Garen_2013_Recall_loop',
 'Garen_2013_Respawn',
 'Garen_2013_Run',
 'Garen_2013_Run_Fast',
 'Garen_2013_Run_Haste',
 'Garen_2013_Run_Spell1',
 'Garen_2013_channel',
 'Garen_2013_channelin',
 'Garen_2013_dance',
 'Garen_2013_dance_loop',
 'Garen_2013_laugh',
 'Garen_2013_spell1',
 'Garen_2013_spell4',
 'Garen_2013_taunt',
 'Garen_Base_spell3_-180',
 'Garen_Base_spell3_-90',
 'Garen_Base_spell3_0',
 'Garen_Base_spell3_180',
 'Garen_Base_spell3_90']